In [1]:
"""
Dedalus script simulating 2D horizontally-periodic Rayleigh-Benard convection.
This script demonstrates solving a 2D Cartesian initial value problem. It can
be ran serially or in parallel, and uses the built-in analysis framework to save
data snapshots to HDF5 files. The `plot_snapshots.py` script can be used to
produce plots from the saved data. It should take about 5 cpu-minutes to run.

For incompressible hydro with two boundaries, we need two tau terms for each the
velocity and buoyancy. Here we choose to use a first-order formulation, putting
one tau term each on auxiliary first-order gradient variables and the others in
the PDE, and lifting them all to the first derivative basis. This formulation puts
a tau term in the divergence constraint, as required for this geometry.

To run and plot using e.g. 4 processes:
    $ mpiexec -n 4 python3 rayleigh_benard.py
    $ mpiexec -n 4 python3 plot_snapshots.py snapshots/*.h5
"""


'\nDedalus script simulating 2D horizontally-periodic Rayleigh-Benard convection.\nThis script demonstrates solving a 2D Cartesian initial value problem. It can\nbe ran serially or in parallel, and uses the built-in analysis framework to save\ndata snapshots to HDF5 files. The `plot_snapshots.py` script can be used to\nproduce plots from the saved data. It should take about 5 cpu-minutes to run.\n\nThe problem is non-dimensionalized using the box height and freefall time, so\nthe resulting thermal diffusivity and viscosity are related to the Prandtl\nand Rayleigh numbers as:\n    kappa = (Rayleigh * Prandtl)**(-1/2)\n    nu = (Rayleigh / Prandtl)**(-1/2)\n\nFor incompressible hydro with two boundaries, we need two tau terms for each the\nvelocity and buoyancy. Here we choose to use a first-order formulation, putting\none tau term each on auxiliary first-order gradient variables and the others in\nthe PDE, and lifting them all to the first derivative basis. This formulation puts\na tau 

In [1]:
import numpy as np
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)

In [2]:
# Parameters
Lx, Lz = 4,1
Nx, Nz = 128, 32
Ra_D= -1.24e5
D_0 = 0
D_H = 2
M_0 = 0
M_H = -6
N_s2=8

Prandtl = 0.7
dealias = 3/2
stop_sim_time = 200
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64

In [3]:
# Bases
coords = d3.CartesianCoordinates('x','z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)

In [4]:
# Fields
p = dist.Field(name='p', bases=(xbasis,zbasis))
D = dist.Field(name='D', bases=(xbasis,zbasis))
M = dist.Field(name='M', bases=(xbasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis))
Z = dist.Field(name='Z', bases=zbasis)
tau_p = dist.Field(name='tau_p')
tau_B1 = dist.Field(name='tau_B1', bases=xbasis)
tau_B2 = dist.Field(name='tau_B2', bases=xbasis)
tau_D1 = dist.Field(name='tau_D1', bases=xbasis)
tau_D2 = dist.Field(name='tau_D2', bases=xbasis)
tau_M1 = dist.Field(name='tau_M1', bases=xbasis)
tau_M2 = dist.Field(name='tau_M2', bases=xbasis)
tau_u1 = dist.VectorField(coords, name='tau_u1', bases=xbasis)
tau_u2 = dist.VectorField(coords, name='tau_u2', bases=xbasis)


# Substitutions

kappa = (Ra_D * Prandtl/((D_0-D_H)*Lz**3))**(-1/2)
nu = (Ra_D / (Prandtl*(D_0-D_H)*Lz**3))**(-1/2)
      
#Kuo_Bretherton Equilibrium
Ra_M = Ra_D*(M_0-M_H)/(D_0-D_H)
G_D=(D_0-D_H)/Lz
G_M=(M_0-M_H)/Lz
Ra_BV=N_s2*Lz**4/(nu*kappa)
print(Ra_M)
print(Ra_BV)

x,z = dist.local_grids(xbasis,zbasis)
Z['g']=z
Z.change_scales(3/2)

ex,ez = coords.unit_vector_fields(dist)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)

B_op = (np.absolute(D - M - N_s2*Z)+ M + D - N_s2*Z)/2

Max = lambda A,B: (abs(A-N_s2*Z-B)+A-N_s2*Z+B)/2
eva = lambda A: A.evaluate()

dz= lambda A: d3.Differentiate(A, coords['z'])
dx= lambda A: d3.Differentiate(A, coords['x'])

ux=u@ex
uz=u@ez

grad_u = d3.grad(u) + ez*lift(tau_u1) # First-order reduction
grad_M = d3.grad(M) + ez*lift(tau_M1) # First-order reduction
grad_D = d3.grad(D) + ez*lift(tau_D1) # First-order reduction

372000.0
496000.00000000006


In [5]:
# Problem
# First-order form: "div(f)" becomes "trace(grad_f)"
# First-order form: "lap(f)" becomes "div(grad_f)"
problem = d3.IVP([p, M, D, u, tau_p, tau_M1, tau_M2, tau_D1, tau_D2, tau_u1, tau_u2], namespace=locals())
problem.add_equation("trace(grad_u) + tau_p= 0")
problem.add_equation("dt(M) - kappa*div(grad_M) + lift(tau_M2) - G_M*uz= - u@grad(M)")
problem.add_equation("dt(D) - kappa*div(grad_D) + lift(tau_D2) - G_D*uz= - u@grad(D)")
problem.add_equation("dt(u) - nu*div(grad_u) + grad(p)  + lift(tau_u2) = - u@grad(u)+ B_op*ez")
problem.add_equation("M(z=0) = M_0")
problem.add_equation("D(z=0) = D_0")
problem.add_equation("dz(u)(z=Lz)@ex = 0")
problem.add_equation("u(z=Lz)@ez = 0")
problem.add_equation("u(z=0) = 0")
problem.add_equation("M(z=Lz) = M_H")
problem.add_equation("D(z=Lz) = D_H")
problem.add_equation("integ(p) = 0") # Pressure gauge

{'LHS': Integrate(Integrate(<Field 5288700368>)),
 'RHS': 0,
 'condition': 'True',
 'tensorsig': (),
 'dtype': numpy.float64,
 'M': 0,
 'L': Integrate(Integrate(<Field 5288700368>)),
 'F': <Field 4775935760>,
 'domain': <dedalus.core.domain.Domain at 0x13b3b3f50>,
 'matrix_dependence': array([ True,  True]),
 'matrix_coupling': array([False,  True])}

In [6]:
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time


2023-09-20 18:06:34,327 subsystems 0/1 INFO :: Building subproblem matrices 1/64 (~2%) Elapsed: 0s, Remaining: 7s, Rate: 9.0e+00/s
2023-09-20 18:06:34,544 subsystems 0/1 INFO :: Building subproblem matrices 7/64 (~11%) Elapsed: 0s, Remaining: 3s, Rate: 2.1e+01/s
2023-09-20 18:06:34,798 subsystems 0/1 INFO :: Building subproblem matrices 14/64 (~22%) Elapsed: 1s, Remaining: 2s, Rate: 2.4e+01/s
2023-09-20 18:06:35,052 subsystems 0/1 INFO :: Building subproblem matrices 21/64 (~33%) Elapsed: 1s, Remaining: 2s, Rate: 2.5e+01/s
2023-09-20 18:06:35,305 subsystems 0/1 INFO :: Building subproblem matrices 28/64 (~44%) Elapsed: 1s, Remaining: 1s, Rate: 2.6e+01/s
2023-09-20 18:06:35,553 subsystems 0/1 INFO :: Building subproblem matrices 35/64 (~55%) Elapsed: 1s, Remaining: 1s, Rate: 2.6e+01/s
2023-09-20 18:06:35,802 subsystems 0/1 INFO :: Building subproblem matrices 42/64 (~66%) Elapsed: 2s, Remaining: 1s, Rate: 2.6e+01/s
2023-09-20 18:06:36,049 subsystems 0/1 INFO :: Building subproblem matri

In [7]:
# Initial condition
D.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
D['g'] *= z * (Lz - z) # Damp noise at walls
D['g'] += (D_H-D_0)*z # Add linear background
M.fill_random('g', seed=28, distribution='normal', scale=1e-3) # Random noise
M['g'] *= z * (Lz - z) # Damp noise at walls
M['g'] += (M_H-M_0)*z # Add linear background

In [8]:
# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.25, max_writes=50)
snapshots.add_tasks(solver.state,layout='g')

In [9]:
# CFL
CFL = d3.CFL(solver, initial_dt=0.0001, cadence=10, safety=0.5, threshold=0.05,
             max_change=1.1, min_change=0.5, max_dt=max_timestep)
CFL.add_velocity(u)

In [10]:
# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property(np.sqrt(u@u)/nu, name='Re')


In [11]:
# Main loop
startup_iter = 10
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            max_Re = flow.max('Re')
            logger.info('Iteration=%i, Time=%e, dt=%e, max(Re)=%f' %(solver.iteration, solver.sim_time, timestep, max_Re))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()

2023-09-20 18:06:54,368 __main__ 0/1 INFO :: Starting main loop
2023-09-20 18:06:55,399 __main__ 0/1 INFO :: Iteration=1, Time=1.000000e-04, dt=1.000000e-04, max(Re)=0.000000
2023-09-20 18:06:55,525 __main__ 0/1 INFO :: Iteration=11, Time=1.100000e-03, dt=1.000000e-04, max(Re)=0.000096
2023-09-20 18:06:55,710 __main__ 0/1 INFO :: Iteration=21, Time=2.200000e-03, dt=1.100000e-04, max(Re)=0.000199
2023-09-20 18:06:55,895 __main__ 0/1 INFO :: Iteration=31, Time=3.410000e-03, dt=1.210000e-04, max(Re)=0.000308
2023-09-20 18:06:56,074 __main__ 0/1 INFO :: Iteration=41, Time=4.741000e-03, dt=1.331000e-04, max(Re)=0.000425
2023-09-20 18:06:56,253 __main__ 0/1 INFO :: Iteration=51, Time=6.205100e-03, dt=1.464100e-04, max(Re)=0.000550
2023-09-20 18:06:56,432 __main__ 0/1 INFO :: Iteration=61, Time=7.815610e-03, dt=1.610510e-04, max(Re)=0.000684
2023-09-20 18:06:56,615 __main__ 0/1 INFO :: Iteration=71, Time=9.587171e-03, dt=1.771561e-04, max(Re)=0.000825
2023-09-20 18:06:56,793 __main__ 0/1 INFO

KeyboardInterrupt: 